In [1]:
# run sentiment analysis for OpenAI API and Azure API

from utils import TestDataset, save_to_csv
from tqdm import tqdm
import pandas as pd 
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI,AzureOpenAI

def inference(prompt, client, model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}], 
            max_tokens=4,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print("!!!!! Error when processing prompt !!!!!")
        print(prompt)
        # print(f"!!!!! {e} !!!!!")
        return None

def batch_inference(batch_data, client, model, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = list(
            tqdm(
                executor.map(lambda x: inference(x, client, model), batch_data),
                total=len(batch_data)
            )
        )
        for future in futures:
            results.append(future)
    return results

In [2]:
# use appropriate dataset for inference
# data_path = "data/train_easy.csv"
# data_path = "data/train_hard.csv"
# data_path = "data/FPB.csv"
data_path = "data/sentimental_analysis/FIQASA/test.csv"

prompt_template = '''Instruction: Analyze the sentiment of this statement extracted from a financial news article. 
Provide your answer as either negative, positive or neutral without anything else.
For instance, The company's stocks plummeted following the scandal. would be classified as negative.
Text: [Input Text]
Response: '''

print("----- Load Dataset -----")
test_dataset = TestDataset(data_path, prompt_template=prompt_template)
batch_data = [test_dataset[i]['sentence'] for i in range(len(test_dataset))]

print("----- Inference Deepseek -----")
deepseek = OpenAI(api_key="sk-619b8461a6e546d5a69c207e9b37ddc2", base_url="https://api.deepseek.com")
deepseek_outputs = batch_inference(batch_data, deepseek, "deepseek-chat", 16)
save_to_csv(data_path, "deepseek", deepseek_outputs)

print("----- Inference Azure GPT-4O-Mini -----")
gpt = AzureOpenAI(api_key="b9135a15c242432cb20ddc43fea3a413", api_version="2023-06-01-preview", azure_endpoint="https://openai-oe.openai.azure.com/")
gpt_outputs = batch_inference(batch_data, gpt, "gpt-4o-mini", 16)
save_to_csv(data_path, "openai", gpt_outputs)

----- Load Dataset -----
----- Inference Deepseek -----


100%|██████████| 235/235 [00:24<00:00,  9.70it/s]


----- Inference Azure GPT-4O-Mini -----


 10%|█         | 24/235 [00:03<00:24,  8.51it/s]

!!!!! Error when processing prompt !!!!!
Instruction: Analyze the sentiment of this statement extracted from a financial news article. 
Provide your answer as either negative, positive or neutral without anything else.
For instance, The company's stocks plummeted following the scandal. would be classified as negative.
Text: $PKT Momentum building for breakout trigger ! Long setup http://stks.co/eXVU
Response: 


100%|██████████| 235/235 [00:14<00:00, 15.72it/s]
